In [ ]:
import numpy as np
from scipy.io import loadmat
import deepxde as dde
import paddle
import matplotlib.pyplot as plt

paddle.set_device('cpu')  # Change to 'gpu' if available

DATA_PATH = r"C:\Users\sauja\Downloads\cylinder_nektar_wake.mat"

def load_training_data(num_points=5000):
    data = loadmat(DATA_PATH)
    X_star = data["X_star"]  # (N, 2)
    t_star = data["t"]       # (T, 1)
    U_star = data["U_star"]  # (N, 2, T)
    P_star = data["p_star"]  # (N, T)

    N = X_star.shape[0]
    T = t_star.shape[0]

    XX = np.tile(X_star[:, 0:1], (1, T))  # (N, T)
    YY = np.tile(X_star[:, 1:2], (1, T))  # (N, T)
    TT = np.tile(t_star.T, (N, 1))        # (N, T)

    UU = U_star[:, 0, :]  # (N, T)
    VV = U_star[:, 1, :]  # (N, T)

    x = XX.flatten()[:, None]
    y = YY.flatten()[:, None]
    t = TT.flatten()[:, None]

    u = UU.flatten()[:, None]
    v = VV.flatten()[:, None]

    data_all = np.hstack((x, y, t, u, v))

    # Filter domain: space and time bounds
    data_filtered = data_all[
        (data_all[:, 2] <= 7) &      # time <= 7
        (data_all[:, 0] >= 1) &      # x >= 1
        (data_all[:, 0] <= 8) &      # x <= 8
        (data_all[:, 1] >= -2) &     # y >= -2
        (data_all[:, 1] <= 2)        # y <= 2
    ]

    idx = np.random.choice(len(data_filtered), size=num_points, replace=False)

    x_train = data_filtered[idx, 0:1]
    y_train = data_filtered[idx, 1:2]
    t_train = data_filtered[idx, 2:3]
    u_train = data_filtered[idx, 3:4]
    v_train = data_filtered[idx, 4:5]

    X = np.hstack((x_train, y_train, t_train))
    Y = np.hstack((u_train, v_train))

    return X, Y

def create_model(X, Y):
    spatial_domain = dde.geometry.Rectangle([1.0, -2.0], [8.0, 2.0])
    time_domain = dde.geometry.TimeDomain(0.0, 7.0)
    geomtime = dde.geometry.GeometryXTime(spatial_domain, time_domain)

    net = dde.nn.FNN([3] + [50]*3 + [2], "tanh", "Glorot normal")

    # Fixed constants (not trainable)
    C1 = dde.Variable(1.0)
    C2 = dde.Variable(0.01)

    def pde(x, y):
        u = y[:, 0:1]
        v = y[:, 1:2]

        u_x = dde.grad.jacobian(y, x, i=0, j=0)
        u_y = dde.grad.jacobian(y, x, i=0, j=1)
        u_t = dde.grad.jacobian(y, x, i=0, j=2)

        v_x = dde.grad.jacobian(y, x, i=1, j=0)
        v_y = dde.grad.jacobian(y, x, i=1, j=1)
        v_t = dde.grad.jacobian(y, x, i=1, j=2)

        u_xx = dde.grad.hessian(y, x, component=0, i=0, j=0)
        u_yy = dde.grad.hessian(y, x, component=0, i=1, j=1)

        v_xx = dde.grad.hessian(y, x, component=1, i=0, j=0)
        v_yy = dde.grad.hessian(y, x, component=1, i=1, j=1)

        eq1 = u_t + C1 * (u * u_x + v * u_y) - C2 * (u_xx + u_yy)
        eq2 = v_t + C1 * (u * v_x + v * v_y) - C2 * (v_xx + v_yy)

        return [eq1, eq2]

    observe_u = dde.icbc.PointSetBC(X, Y[:, 0:1], component=0)
    observe_v = dde.icbc.PointSetBC(X, Y[:, 1:2], component=1)

    data = dde.data.TimePDE(
        geomtime,
        pde,
        [observe_u, observe_v],
        num_domain=500,
        num_boundary=200,
    )

    model = dde.Model(data, net)
    return model

def plot_contours(model):
    # Generate grid points for plotting at t=7 (final time)
    x = np.linspace(1, 8, 100)
    y = np.linspace(-2, 2, 100)
    X_grid, Y_grid = np.meshgrid(x, y)
    t_grid = 7 * np.ones_like(X_grid)

    points = np.vstack((X_grid.flatten(), Y_grid.flatten(), t_grid.flatten())).T

    pred = model.predict(points)
    u_pred = pred[:, 0]
    v_pred = pred[:, 1]

    # Velocity magnitude
    vel_mag = np.sqrt(u_pred**2 + v_pred**2)

    # Calculate vorticity: ω = dv/dx - du/dy
    # Use automatic differentiation for derivatives

    # For this we need to run model.predict with gradient support:
    # deepxde has model.predict with grad operator, but Paddle backend may need special handling.

    # So we use dde.grad.jacobian with Paddle backend:

    import paddle

    x_paddle = paddle.to_tensor(points, dtype='float32', stop_gradient=False)
    u_paddle = model.net(x_paddle)[:, 0:1]
    v_paddle = model.net(x_paddle)[:, 1:2]

    du_dy = dde.grad.jacobian(u_paddle, x_paddle, i=0, j=1).numpy()
    dv_dx = dde.grad.jacobian(v_paddle, x_paddle, i=0, j=0).numpy()

    vorticity = dv_dx.flatten() - du_dy.flatten()

    # Reshape for plotting
    vel_mag_grid = vel_mag.reshape(X_grid.shape)
    vorticity_grid = vorticity.reshape(X_grid.shape)

    # Plot velocity magnitude contour
    plt.figure(figsize=(12, 5))

    plt.subplot(1, 2, 1)
    cp1 = plt.contourf(X_grid, Y_grid, vel_mag_grid, levels=50, cmap='viridis')
    plt.colorbar(cp1)
    plt.title('Velocity Magnitude |V| at t=7')
    plt.xlabel('x')
    plt.ylabel('y')
    plt.axis('scaled')

    # Plot vorticity contour
    plt.subplot(1, 2, 2)
    cp2 = plt.contourf(X_grid, Y_grid, vorticity_grid, levels=50, cmap='plasma')
    plt.colorbar(cp2)
    plt.title('Vorticity ω at t=7')
    plt.xlabel('x')
    plt.ylabel('y')
    plt.axis('scaled')

    plt.tight_layout()
    plt.show()


# --- Run Training and Plotting ---

X_train, Y_train = load_training_data(num_points=5000)
model = create_model(X_train, Y_train)

model.compile("adam", lr=1e-3)
model.train(epochs=1000, display_every=200)

plot_contours(model)
